In [3]:
from torchmetrics.functional import structural_similarity_index_measure
from torchmetrics.image import PeakSignalNoiseRatio
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
import re 
import json 
import os
from PIL import Image, ImageDraw, ImageFont
from torchvision.transforms import ToTensor

psnr_metric = PeakSignalNoiseRatio(data_range=1.0)
lpips_metric = LearnedPerceptualImagePatchSimilarity()
ssim_metric = structural_similarity_index_measure

ORIGINAL_IMAGES_PATH = r'C:\Stuff\Results\renders\lego_original_images'
RENDERS_PATH = r"C:\Stuff\Results\renders"

def get_timestamp(path):
    f = open(path)
    data =  json.load(f)
    ckpt_path = data['checkpoint']
    pattern = r'\d{4}-\d{2}-\d{2}_\d{6}'
    match = re.search(pattern, ckpt_path)
    timestamp = match.group()
    return timestamp

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return ToTensor()(image)


In [5]:
psnrs, lpips_vals, ssims = [], [], []
exp_metrics = []

PATH_JSONS_FOLDERS = r"C:\Stuff\Results\jsons"
# jsons_folder = r"C:\Stuff\Results\jsons\jsons_nerfacto_lego_opt_off"
# renders_folder = r"C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off"

for jsons_folder in os.listdir(PATH_JSONS_FOLDERS):
    if 'lego' in jsons_folder:
        renders_folder = jsons_folder.replace('jsons', 'renders')
        for filename in os.listdir(jsons_folder):
            if filename.endswith(".json") and 'position' in filename:
                json_filename = filename
                json_path = os.path.join(jsons_folder, filename)
                timestamp = get_timestamp(json_path)

                run_path = os.path.join(renders_folder, timestamp)
                print(run_path)
                if os.path.isdir(run_path):
                    psnrs, lpips_vals, ssims = [], [], []

                    for filename in os.listdir(run_path):
                        if filename.endswith('img.jpg') and not filename.startswith('crop_degrated_'):
                            base_name = filename.split('-')[0]
                            degraded_filename = f'crop_degrated_{base_name}-img.jpg'
                            degraded_filepath = os.path.join(run_path, degraded_filename)

                            degraded_image = load_image(degraded_filepath)
                            original_image = load_image(os.path.join(ORIGINAL_IMAGES_PATH, base_name + '-img.jpg'))

                            original_image = original_image.unsqueeze(0)  # Add batch dimension
                            degraded_image = degraded_image.unsqueeze(0)  # Add batch dimension

                            psnr = psnr_metric(original_image, degraded_image)
                            ssim = ssim_metric(original_image, degraded_image)
                            # lpips = lpips_metric(original_image, degraded_image)
                            psnrs.append(psnr.item())
                            ssims.append(ssim.item())
                            # lpips_vals.append(lpips.item())
                            lpips_vals.append(0) 

                    # print(run_path)
                    exp_metrics.append({
                        'name': json_filename[:-5],
                        'run': timestamp,
                        'PSNR': psnrs,
                        'SSIM': ssims,
                        'LPIPS': lpips_vals
                    })
        for run_metrics in exp_metrics:
            print(f"Metrics for run {run_metrics['run']}:")
            print(f"Name {run_metrics['name']}:")
            print("PSNR values:", run_metrics['PSNR'])
            print("SSIM values:", run_metrics['SSIM'])
            print("LPIPS values:", run_metrics['LPIPS'])
            print("\n")

C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_213015


c:\Users\danko\miniconda3\envs\nerfstudio\lib\site-packages\torchmetrics\utilities\prints.py:70: FutureWarning: Importing `spectral_angle_mapper` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `spectral_angle_mapper` from `torchmetrics.image` instead.
  _future_warning(


C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_214155
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_215332
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_220511
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_221651
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_222828
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_224004
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_225134
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_230315
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_231455
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_232631
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_233808
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-02_234940
C:\Stuff\Results\renders\renders_nerfacto_lego_opt_off\2024-03-03_000114
Metrics for run 2024-03-02_213015:
Name lego-positi